In [2]:
import numpy as np
import itertools
import math

In [3]:
np.set_printoptions(linewidth=200)

In [4]:
def choose_ij(vals, i, j):
    return vals[i] / vals[j]

In [5]:
def choose_both(vals, i, j):
    return choose_ij(vals, i, j) + choose_ij(vals, j, i)

In [6]:
def f_first(vals):
    prod = np.complex128(1)
    for j in range(len(vals)):
        for k in range(j+1, len(vals)):
            prod *= choose_both(vals, j, k)

    return prod

In [7]:
def f_second(vals):
    A = np.zeros([len(vals), len(vals)], dtype=np.complex128)
    for j in range(len(vals)):
        for k in range(len(vals)):
            if j == k:
                continue
            A[j,k] = -1 * choose_ij(vals, j, k) / choose_both(vals, j, k)

    for i in range(len(vals)):
        # check if this is row or col sum
        A[i, i] = -1 * A[i, :].sum()

    return np.linalg.det(A[:-1, :-1])

In [8]:
def f(vals):
    return f_first(vals) * f_second(vals)

In [9]:
def count_cycles(n, m):
    roots = np.roots([1] + [0]*(m-1) + [-1])

    s = np.complex128()

    for x in itertools.product(roots, repeat=n-1):
        s += f(x + (np.complex128(1),))
    
    return s / m ** (n-1)

In [10]:
# So - our calculation has given us both the constant term - and also the terms where the exponents are all in {0, -6, 6}
# The exponents of the x_i are calculating the "net out degree". So note that the only vertex that can have net out degree is the root vertex of our tree - v_n
# Note also that the total degree of each term is zero. So therefore these terms must all be of the form x_i^6*x_n*-6 - i!= n
# We could try and calculate this term directly (and multiply by n-1 for the n-1 choices for i at the end)
# Wlog take i = n-1
# We want to force our graph to have directed edges E_n-1,i and E_i,n. 
# So we can take an adjusted f_first with all these coefficients
# Then in our matrix we could fill things as normal but put a zero if E_i,n-1 or E_n,i is in the tree, and put a 1 where E_n-1,i or E_i,n in the tree
# This also lets us factor x_i, x_n out of our formula. Taking these two out both adds and removes 1 from each degree putting us back where we started

In [11]:
def f_first_restricted(vals, forced_values: dict[dict[bool]]):
    prod = np.complex128(1)
    for j in range(len(vals)):
        for k in range(j+1, len(vals)):
            if j in forced_values or k in forced_values:
                continue
            else:
                prod *= choose_both(vals, j, k)

    for f_vals in forced_values.values():
        for second_vertex, points_to_second in f_vals.items():
            if points_to_second:
                prod /= vals[second_vertex]
            else:
                prod *= vals[second_vertex]

    return prod

In [12]:
def f_second_restricted(vals, forced_values: dict[dict[bool]]):
    n = len(vals)
    A = np.zeros([n, n], dtype=np.complex128)
    for j in range(len(vals)):
        for k in range(len(vals)):
            if j == k:
                continue

            if j in forced_values or k in forced_values:
                continue

            A[j,k] = -1 * choose_ij(vals, j, k) / choose_both(vals, j, k)

    for first_vertex, dirs in forced_values.items():
        for second_vertex, points_to_second in dirs.items():
            A[first_vertex, second_vertex] = -int(points_to_second)
            A[second_vertex, first_vertex] = -int(not points_to_second)
                
    for i in range(n):
        # check if this is row or col sum
        A[i, i] = -1 * A[i, :].sum()

    # return A
    
    return np.linalg.det(A[:-1, :-1])

In [13]:
def f_restricted(vals, forced_values: dict[dict[bool]]):
    return f_first_restricted(vals, forced_values) * f_second_restricted(vals, forced_values)

In [43]:
n = 5 
m = 3
roots = np.roots([1] + [0]*(m-1) + [-1])

s = np.complex128()

for x in itertools.product(roots, repeat=n-2):
    s += f_restricted((1,) + x + (np.complex128(1),), {0: {1: True, 2: True, 3: False, 4:False}})

s / m ** (n-2)

np.complex128(43.99999999999997+0j)

In [136]:
{i: i%2 == 0 for i in range (1, n)}

{1: False, 2: True, 3: False, 4: True}

In [67]:
def calculate_efficient(n, m, forcings):
    # n = 7
    # m = 5
    roots = np.roots([1] + [0]*(m-1) + [-1])
    
    # hardcodes = 
    
    s = np.complex128()
    
    for x in itertools.product(roots, repeat=n-len(forcings)-1):
        s += f_restricted((1,) + x + (np.complex128(1),), forcings)
    
    return s * math.comb(n-1, (n-1)//2) / m ** (n-2)

In [69]:
calculate_efficient(7, 5, {0: {i: i%2 == 0 for i in range (1, 7)}})

np.complex128(1015440.0000000033-5.364418029785157e-10j)

In [70]:
count_cycles(7, 5)

np.complex128(1015439.999999994-3.883838653564453e-10j)

In [66]:
calculate_efficient(5, 3, {0: {i: i%2 == 0 for i in range (1, 5)}, 1: {0:True, 2:False, 3:True, 4:False}})

np.complex128(-2.5263741715914672e-14-5.921189464667501e-15j)

In [49]:
calculate_efficient(1, 5, 3)

np.complex128(263.9999999999999+0j)

In [148]:
507720 * 2

1015440

In [134]:
264 / 88

3.0

In [ ]:
# Ok - let's play the K9 game
# We're supposed to use M=5 
# Suppose we use M=3...
# We pick up powers of X^6
# Let's say for now we sum the final root over m=5...
# That means we can't have any powers of 6 in the root vertex
# Every vertex needs to have at least one out node
# So let's look at the in counts... Say vertex 1 has a power of -6... That means exactly 7 ins and 1 out
# The out has to be going to the root vertex...
# So this is actually super constrained

In [50]:
def jack_offset_f(vals):
    return f_first(vals) * f_second_mod(vals)

In [54]:
def calc_jack_offset(n, m):
    roots = np.roots([1] + [0]*(m-1) + [-1])
    
    s = np.complex128()
    
    for x in itertools.product(roots, repeat=n-3):
        s += jack_offset_f(x + (np.complex128(1),))
    
    return (n-1) * s / m ** (n-3)

In [77]:
n = 7
m1 = 3
m2 = 5

roots1 = np.roots([1] + [0]*(m1-1) + [-1])
roots2 = np.roots([1] + [0]*(m2-1) + [-1])

s = np.complex128()

for x in itertools.product(roots1, repeat=n-2):
    for r2 in roots2:
        s += f(x + (1, r2))
    
s / m2 / m1 ** (n-2)

np.complex128(1015440.0000000001+0j)

In [76]:
n = 9
m1 = 3
m2 = 5

roots1 = np.roots([1] + [0]*(m1-1) + [-1])
roots2 = np.roots([1] + [0]*(m2-1) + [-1])

s = np.complex128()

for x in itertools.product(roots1, repeat=n-1):
    for r2 in roots2:
        s += f(x + (r2,))
    
s / m2 / m1 ** (n-1)

np.complex128(159770231039.9981-9.525986892242037e-07j)

In [74]:
#K9 estimate:
159770231040

#K9 real value:
90449251200


3.0

In [ ]:
3046320
1015440
